### **🎯 가설검정의 이해**
**통계적 가설검정**
- 귀무가설(H₀) : 기존에 알려진 사실
- 대립가설(H₁) : 입증하고자 하는 가설

**가설검정 프로세스**
1. 통계적 가설설정
- 검정하고자 하는 가설 설정
2. 유의수준 설정
- 유의수준 : 귀무가설을 기각하기 위해 필요한 기준값 (보통 0.05, 0.01)
3. 검정 통계량 및 유의확률(p-value) 계산
- 검정 통계량 : 가설을 평가하기 위해 사용되는 값
4. 결과 도출
- p-value < 유의수준 : 귀무가설 기각
- p-value > 유의수준 : 귀무가설 채택


🧪 **평균 비교 가설검정 요약표**

✅ 단일 표본 검정 (1개 집단 vs 모집단 평균)

| 정규성 | 사용 검정 | 함수 예시 |
|--------|------------|-------------|
| 만족 | 단일표본 t-검정 | `ttest_1samp()` |
| 불만족 | 윌콕슨 부호순위 검정 | `wilcoxon()`|

✅ 대응 표본 검정 (두 집단, 짝지어진 데이터)

| 정규성 | 사용 검정 | 함수 예시 |
|--------|------------|-------------|
| 만족 | 대응표본 t-검정 | `ttest_rel()` |
| 불만족 | 윌콕슨 부호순위 검정 | `wilcoxon()` |
| 정규성 검정 방법 | Shapiro-Wilk 정규성 검정 | `shapiro()` |

✅ 독립 표본 검정 (두 집단, 독립된 데이터)

| 조건 | 정규성 | 분산 동질성 | 사용 검정 | 함수 예시 |
|------|--------|--------------|------------|-------------|
| 일반적 케이스 | 만족 | 만족 | 독립표본 t-검정 | `ttest_ind()` |
| Welch 케이스 | 만족 | 불만족 | Welch의 t-검정 | `ttest_ind(equal_var=False)` |
| 비모수 | 불만족 | - | Mann-Whitney U 검정 | `mannwhitneyu()` |
| 정규성 검정 | - | - | Shapiro-Wilk | `shapiro()` |
| 분산 동질성 검정 | - | - | Levene or Bartlett 검정 | `levene()`, `bartlett()` |
---
### **✅ 단일 표본 검정**
#### **단일 표본 t-검정**
- 어떤 집단의 평균이 특정 값과 유의미하게 다른지 검정.
- 정규성을 만족할 때


`ttest_1samp(a, popmean, alternative)`
- a : 모집단에서 뽑은 표본 데이터
- popmean : 비교하려는 모집단의 평균 또는 기댓값
- alternative : 대립가설 정의 (기본적으로 양측 검정 수행)
    - 양측 검정 : `two-sided`
    - 단측 검정
        - `greater` : 귀무가설의 평균보다 크다.
        - `less` : 귀무가설의 평균보다 작다.


μ : 모집단의 평균 / 
μ₀ : 기존에 알려진 값

In [3]:
import pandas as pd

df = pd.DataFrame({
    'weights':[122, 121, 120, 119, 125, 115, 121, 118, 117, 127,
           123, 129, 119, 124, 114, 126, 122, 124, 121, 116,
           120, 123, 127, 118, 122, 117, 124, 125, 123, 121],
})

In [7]:
from scipy import stats

t_stat, p_value = stats.ttest_1samp(df['weights'], 120)
print('t_stat:', t_stat)
print('p-value:', p_value)

stats.ttest_1samp(df['weights'], 120)

t_stat: 2.1155384372682344
p-value: 0.043092957066609296


TtestResult(statistic=2.1155384372682344, pvalue=0.043092957066609296, df=29)

In [12]:
print(stats.ttest_1samp(df['weights'], 120, alternative='greater'))
print(stats.ttest_1samp(df['weights'], 120, alternative='less'))

TtestResult(statistic=2.1155384372682344, pvalue=0.021546478533304648, df=29)
TtestResult(statistic=2.1155384372682344, pvalue=0.9784535214666953, df=29)


#### **Wilcoxon의 부호 순위 검정**

1. `shapiro()` : 샤피로-윌크 검정, 주어진 데이터 샘플이 정규 분포를 따르는지 검정
2. `wilcoxon()` : 데이터가 정규성 가정을 만족하지 않을 때 (비모수 검정)

In [17]:
# H₀ : 주어진 데이터는 정규분포를 따른다.
# H₁ :            "         따르지 않는다.
stats.shapiro(df['weights'])
# p-value가 0.05보다 크므로 대립가설 채택
# 주어진 데이터 샘플은 정규성을 만족하지 않음
# -> 윌콕슨의 부호 순위 검정

ShapiroResult(statistic=0.9877707598413207, pvalue=0.9746408136556016)

In [18]:
# H₀ : 중앙값이 120이다.
# H₁ :   "    120보다 작다.
# 비교하는 중앙값 빼줘야 함.
stats.wilcoxon(df['weights'] - 120, alternative='less')

WilcoxonResult(statistic=288.0, pvalue=0.9738893363039633)

---
### **✅ 대응 표본 검정**
동일한 그룹(집단)에 대해 시간차를 두고 두 번의 측정 결과를 비교
- ex) 신약의 효과가 유의미한지 확인하기 위해, 1개의 집단을 사전/사후 측정
- `ttest_rel(a, b, alternative)`
     - a : 첫번째 모집단에서 뽑은 표본 데이터
     - b : 두번째 모집단에서 뽑은 표본 데이터

In [31]:
import pandas as pd

df = pd.DataFrame({
    'before':[85, 90, 92, 88, 86, 89, 83, 87],
    'after':[85.5,89.9,92.6,89.5,85.8,88.8,84.6,87.8]
})

In [32]:
from scipy import stats

stats.ttest_rel(df['before'], df['after'], alternative='less')

TtestResult(statistic=-2.2127749675452324, pvalue=0.03127028733756238, df=7)

정규 분포를 따른다고 명시적으로 언급되지 않았을 경우, 정규성 검정 실시


1. 샤피로-윌크 검정 : 데이터의 정규성을 평가
2. 윌콕슨 부호 순위 검정 : 데이터가 정규성을 만족하지 않을 때 

In [33]:
import pandas as pd

df = pd.DataFrame({
    'before':[85, 90, 92, 88, 86, 89, 83, 87],
    'after':[86, 92, 94, 89, 84, 90, 84, 88]
})

In [29]:
from scipy import stats

print(stats.shapiro(df['after'] - df['before']))
print(stats.wilcoxon(df['after'] - df['before'], alternative='greater'))

ShapiroResult(statistic=0.6886147375920879, pvalue=0.0016734051223900109)
WilcoxonResult(statistic=29.0, pvalue=0.07421875)


---
### **✅ 독립 표본 검정**
두 그룹(표본)간의 평균이 서로 다름을 판단하는 통계 방법
- `ttest_ind(a, b, alternative, equal_var)`
    - `equal_var`
        - True : 두 모집단의 분산이 같다고 가정
        - False : 두 모집단의 분산이 다르다고 가정

In [44]:
class1 = [85, 90, 92, 88, 86, 89, 83, 87]
class2 = [80, 82, 88, 85, 84]     

In [36]:
from scipy import stats

stats.ttest_ind(class1, class2, equal_var=False)

TtestResult(statistic=2.1818699281825236, pvalue=0.059589330071355334, df=8.272682358753572)

두 집단의 데이터가 등분산성을 갖는지 확인해야 함.
1. `shaprio()` : 정규성 검정
    - 데이터가 정규성을 만족하지 않을 경우, `mannwhitneyu()` 검정 실시.
2. `levene()` : 등분산성 검정 

모수 검정



H₀ : 반별 수학 평균 점수는 같다.
H₁ : class2의 수학 평균 점수가 더 높다.
 

In [43]:
# class1과 class2의 평균 점수 차이가 있는지 가설검정
class1 = [85, 90, 92, 88, 86, 89, 83, 87]
class2 = [80, 82, 88, 85, 84]

In [38]:
from scipy import stats

print(stats.shapiro(class1)) # 정규성 검정
print(stats.shapiro(class2))

ShapiroResult(statistic=0.9981893537736595, pvalue=0.999986994137081)
ShapiroResult(statistic=0.9917398436295009, pvalue=0.9854182266624983)


In [40]:
print(stats.levene(class1, class2)) # 등분산성 검정

LeveneResult(statistic=0.0027925869510027727, pvalue=0.958802951766629)


In [41]:
stats.ttest_ind(class1, class2, alternative='less', equal_var=True)

TtestResult(statistic=2.2108140580092237, pvalue=0.9754257110537391, df=11.0)

비모수 검정

In [46]:
class1 = [85, 90, 92, 88, 86, 89, 83, 87]
class2 = [80, 82, 88, 85, 130]

In [47]:
from scipy import stats

print(stats.shapiro(class1))
print(stats.shapiro(class2))

ShapiroResult(statistic=0.9981893537736595, pvalue=0.999986994137081)
ShapiroResult(statistic=0.6880497349322277, pvalue=0.007151570728885509)


In [48]:
print(stats.mannwhitneyu(class1, class2, alternative='less'))

MannwhitneyuResult(statistic=26.0, pvalue=0.8299904236851448)
